In [1]:
import sys

In [2]:
sys.path.append('..')

In [3]:
from jamo import j2hcj, h2j
import pandas as pd
import numpy as np
from konlpy.tag import Twitter

from sklearn.metrics import accuracy_score, f1_score
twitter = Twitter()

/Users/1003874/anaconda/envs/lstm/lib/python3.6/site-packages/konlpy/tag/_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


In [4]:
from tensorflow.python.ops.rnn import bidirectional_dynamic_rnn as bi_rnn
from tensorflow.contrib.rnn import BasicLSTMCell
from utils.prepare_data import *
import time
from utils.model_helper import *

/Users/1003874/anaconda/envs/lstm/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [5]:
def testcase_shuffle_data_load():
    df = pd.read_csv('/Users/1003874/bsdev/Text-classification-with-CNN-RNN-with-Tensorflow/Ch01_Data_load/data/preprocessed_mart_digi_speech_act.csv')
    df['clean_text'] = df['clean_text'].apply(lambda x:x.lower())
    s = df['class'].value_counts()
    cols = s[s>50].index.tolist()

    sub_list = []
    for col in cols:
        sub_list.append(df[df['class'] == col])
    df = pd.concat(sub_list, axis=0)
    df = df.sample(len(df), random_state=10).reset_index(drop=True)

    added_df = pd.read_csv('/Users/1003874/bsdev/Text-classification-with-CNN-RNN-with-Tensorflow/Ch01_Data_load/data/added_dataset.csv',sep='\t')
    added_df['clean_text'] = added_df['sent'].apply(lambda x:x.lower())

    # class 제거
    added_df = added_df[added_df['class'].apply(lambda x: True if x in cols else False)]
    added_df.info()

    # duplicated sent 제거
    added_df = added_df[added_df['clean_text'].apply(lambda x: False if x in df['clean_text'].tolist() else True)]
    added_df.info()

    # train_df merge

    size = len(df)
    rate = 0.2
    train_df = df.iloc[:int(size * (1 - rate))]
    test_df = df.iloc[int(size * (1 - rate)):]

    all_df = pd.concat([train_df[['clean_text', 'class']],
                        test_df[['clean_text', 'class']],
                        added_df[['clean_text', 'class']]])

    all_df = all_df.sample(len(all_df), random_state=10)
    all_df.reset_index(drop=True, inplace=True)

    label_df = pd.get_dummies(all_df['class'])

    train_df = all_df.iloc[1062:]
    test_df = all_df.iloc[:1062]

    cols = label_df.columns

    print(len(train_df), len(test_df))

    TRAIN_DOC, TRAIN_LABEL, TEST_DOC, TEST_LABEL = \
        train_df['clean_text'].values, label_df.loc[train_df.index].values, test_df['clean_text'].values, label_df.loc[
            test_df.index].values

    return TRAIN_DOC, TRAIN_LABEL, TEST_DOC, TEST_LABEL, cols

In [6]:
def testcase_add_data_load():
    df = pd.read_csv('/Users/1003874/bsdev/Text-classification-with-CNN-RNN-with-Tensorflow/Ch01_Data_load/data/preprocessed_mart_digi_speech_act.csv')
    df['clean_text'] = df['clean_text'].apply(lambda x:x.lower())
    s = df['class'].value_counts()
    cols = s[s>50].index.tolist()

    sub_list = []
    for col in cols:
        sub_list.append(df[df['class'] == col])
    df = pd.concat(sub_list, axis=0)
    df = df.sample(len(df), random_state=10).reset_index(drop=True)

    added_df = pd.read_csv(
        '/Users/1003874/bsdev/Text-classification-with-CNN-RNN-with-Tensorflow/Ch01_Data_load/data/added_dataset.csv',
        sep='\t')
    added_df['clean_text'] = added_df['sent'].apply(lambda x:x.lower())

    # class 제거
    added_df = added_df[added_df['class'].apply(lambda x: True if x in cols else False)]
    added_df.info()

    # duplicated sent 제거
    added_df = added_df[added_df['clean_text'].apply(lambda x: False if x in df['clean_text'].tolist() else True)]
    added_df.info()

    # train_df merge

    size = len(df)
    rate = 0.2
    train_df = df.iloc[:int(size * (1 - rate))]
    test_df = df.iloc[int(size * (1 - rate)):]
    train_df['setlabel'] = 1
    test_df['setlabel'] = 0
    added_df['setlabel'] = 2

    all_df = pd.concat([train_df[['clean_text', 'class', 'setlabel']],
                        test_df[['clean_text', 'class', 'setlabel']],
                        added_df[['clean_text', 'class','setlabel']]])

    all_df.reset_index(drop=True, inplace=True)

    label_df = pd.get_dummies(all_df['class'])

    train_df = all_df[all_df['setlabel'] != 0]
    test_df = all_df[all_df['setlabel'] == 0]

    cols = label_df.columns

    print(len(train_df), len(test_df))

    TRAIN_DOC, TRAIN_LABEL, TEST_DOC, TEST_LABEL = \
        train_df['clean_text'].values, label_df.loc[train_df.index].values, test_df['clean_text'].values, label_df.loc[
            test_df.index].values

    return TRAIN_DOC, TRAIN_LABEL, TEST_DOC, TEST_LABEL, cols

In [7]:
def digi_data_load():
    df = pd.read_csv('/Users/1003874/bsdev/Text-classification-with-CNN-RNN-with-Tensorflow/Ch01_Data_load/data/preprocessed_mart_digi_speech_act.csv')
    df['clean_text'] = df['clean_text'].apply(lambda x:x.lower())
    s = df['class'].value_counts()
    cols = s[s>50].index.tolist()

    sub_list = []
    for col in cols:
        sub_list.append(df[df['class'] == col])
    df = pd.concat(sub_list, axis=0)
    df = df.sample(len(df), random_state=10).reset_index(drop=True)

    size = len(df)
    rate = 0.2
    train_df = df.iloc[:int(size*(1-rate))]
    test_df = df.iloc[int(size*(1-rate)):]
    label_df = pd.get_dummies(df['class'])
    cols = label_df.columns
    print(len(train_df), len(test_df))

    TRAIN_DOC, TRAIN_LABEL, TEST_DOC, TEST_LABEL = \
        train_df['clean_text'].values, label_df.loc[train_df.index].values, test_df['clean_text'].values, label_df.loc[test_df.index].values

    return TRAIN_DOC, TRAIN_LABEL, TEST_DOC, TEST_LABEL, cols

In [8]:
class ABLSTM(object):
    def __init__(self, config):
        self.max_len = config["max_len"]
        self.hidden_size = config["hidden_size"]
        self.vocab_size = config["vocab_size"]
        self.embedding_size = config["embedding_size"]
        self.n_class = config["n_class"]
        self.learning_rate = config["learning_rate"]

        # placeholder
        self.x = tf.placeholder(tf.int32, [None, self.max_len])
        self.label = tf.placeholder(tf.int32, [None])
        self.keep_prob = tf.placeholder(tf.float32)

    def build_graph(self):
        print("building graph")
        # Word embedding
        embeddings_var = tf.Variable(tf.random_uniform([self.vocab_size, self.embedding_size], -1.0, 1.0),
                                     trainable=True)
        batch_embedded = tf.nn.embedding_lookup(embeddings_var, self.x)

        rnn_outputs, _ = bi_rnn(BasicLSTMCell(self.hidden_size),
                                BasicLSTMCell(self.hidden_size),
                                inputs=batch_embedded, dtype=tf.float32)

        fw_outputs, bw_outputs = rnn_outputs

        W = tf.Variable(tf.random_normal([self.hidden_size], stddev=0.1))
        H = fw_outputs + bw_outputs  # (batch_size, seq_len, HIDDEN_SIZE)
        M = tf.tanh(H)  # M = tanh(H)  (batch_size, seq_len, HIDDEN_SIZE)

        self.alpha = tf.nn.softmax(tf.reshape(tf.matmul(tf.reshape(M, [-1, self.hidden_size]),
                                                        tf.reshape(W, [-1, 1])),
                                              (-1, self.max_len)))  # batch_size x seq_len
        r = tf.matmul(tf.transpose(H, [0, 2, 1]),
                      tf.reshape(self.alpha, [-1, self.max_len, 1]))
        r = tf.squeeze(r)
        h_star = tf.tanh(r)  # (batch , HIDDEN_SIZE

        h_drop = tf.nn.dropout(h_star, self.keep_prob)

        # Fully connected layer（dense layer)
        FC_W = tf.Variable(tf.truncated_normal([self.hidden_size, self.n_class], stddev=0.1))
        FC_b = tf.Variable(tf.constant(0., shape=[self.n_class]))
        y_hat = tf.nn.xw_plus_b(h_drop, FC_W, FC_b)

        self.loss = tf.reduce_mean(
            tf.nn.sparse_softmax_cross_entropy_with_logits(logits=y_hat, labels=self.label))

        # prediction
        self.prediction = tf.argmax(tf.nn.softmax(y_hat), 1)

        # optimization
        loss_to_minimize = self.loss
        tvars = tf.trainable_variables()
        gradients = tf.gradients(loss_to_minimize, tvars, aggregation_method=tf.AggregationMethod.EXPERIMENTAL_TREE)
        grads, global_norm = tf.clip_by_global_norm(gradients, 1.0)

        self.global_step = tf.Variable(0, name="global_step", trainable=False)
        self.optimizer = tf.train.AdamOptimizer(learning_rate=self.learning_rate)
        self.train_op = self.optimizer.apply_gradients(zip(grads, tvars), global_step=self.global_step,
                                                       name='train_step')
        print("graph built successfully!")


In [10]:
TRAIN_DOC, TRAIN_LABEL, TEST_DOC, TEST_LABEL, cols = digi_data_load()
#TRAIN_DOC, TRAIN_LABEL, TEST_DOC, TEST_LABEL, cols = testcase_add_data_load()
#TRAIN_DOC, TRAIN_LABEL, TEST_DOC, TEST_LABEL, cols = testcase_shuffle_data_load()

4244 1062


In [12]:
train_intent = np.apply_along_axis(np.argmax,1,TRAIN_LABEL)
test_intent = np.apply_along_axis(np.argmax,1,TEST_LABEL)

In [14]:
%%time
train_doc = TRAIN_DOC.tolist()
test_doc = TEST_DOC.tolist()

#train_doc = pd.Series(list(map(lambda x: ' '.join(twitter.morphs(x)), train_doc)))
#test_doc = pd.Series(list(map(lambda x: ' '.join(twitter.morphs(x)), test_doc)))
train_doc = pd.Series(list(map(lambda x: j2hcj(h2j(x)), train_doc)))\
    .apply(lambda x: ' '.join(list(x.replace(' ','ⓢ'))))
test_doc = pd.Series(list(map(lambda x: j2hcj(h2j(x)), test_doc)))\
    .apply(lambda x: ' '.join(list(x.replace(' ','ⓢ'))))

CPU times: user 745 ms, sys: 8.09 ms, total: 753 ms
Wall time: 814 ms


In [15]:
y_train = pd.Series(train_intent)
y_test = pd.Series(test_intent)

In [16]:
x_train, x_test, tokenizer = \
        data_preprocessing_v2(train_doc, test_doc, max_len=128,max_words=500)
vocab_size= tokenizer.num_words

In [17]:
tokenizer.word_docs

{'0': 1,
 '1': 84,
 '2': 3,
 '4': 2,
 '5': 2,
 'a': 82,
 'b': 1,
 'c': 6,
 'd': 4,
 'e': 6,
 'f': 5,
 'g': 1,
 'h': 1,
 'i': 38,
 'k': 31,
 'l': 4,
 'm': 8,
 'n': 4,
 'o': 20,
 'p': 57,
 'q': 7,
 'r': 4,
 's': 64,
 't': 25,
 'u': 3,
 'v': 27,
 'w': 1,
 'y': 25,
 'ⓢ': 3544,
 'ㄱ': 2808,
 'ㄲ': 234,
 'ㄴ': 3476,
 'ㄶ': 76,
 'ㄷ': 1966,
 'ㄸ': 325,
 'ㄹ': 2232,
 'ㄺ': 1,
 'ㅀ': 6,
 'ㅁ': 2268,
 'ㅂ': 1712,
 'ㅃ': 34,
 'ㅄ': 113,
 'ㅅ': 2218,
 'ㅆ': 628,
 'ㅇ': 3829,
 'ㅈ': 2046,
 'ㅉ': 51,
 'ㅊ': 682,
 'ㅋ': 607,
 'ㅌ': 516,
 'ㅍ': 1097,
 'ㅎ': 1788,
 'ㅏ': 3430,
 'ㅐ': 1322,
 'ㅑ': 152,
 'ㅒ': 6,
 'ㅓ': 2282,
 'ㅔ': 1571,
 'ㅕ': 906,
 'ㅖ': 95,
 'ㅗ': 2234,
 'ㅘ': 540,
 'ㅙ': 129,
 'ㅚ': 434,
 'ㅛ': 1610,
 'ㅜ': 2084,
 'ㅝ': 354,
 'ㅞ': 10,
 'ㅟ': 229,
 'ㅠ': 93,
 'ㅡ': 2082,
 'ㅢ': 191,
 'ㅣ': 2619}

In [18]:
x_dev = x_test
y_dev = y_test
dev_size = len(x_dev)
test_size = len(x_test)
print("Validation Size: ", dev_size)

config = {
    "max_len": 128,
    "hidden_size": 64,
    "vocab_size": vocab_size,
    "embedding_size": 64,
    "n_class": 33,
    "learning_rate": 1e-3,
    "batch_size": 4,
    "train_epoch": 50
}

classifier = ABLSTM(config)
classifier.build_graph()

Validation Size:  1062
building graph
graph built successfully!


In [22]:
for x_batch, y_batch in fill_feed_dict(x_dev, y_dev,1062):
    print(y_batch)

495     23
372     14
20       5
585     20
5        8
128      5
958     22
602     23
447      4
158     17
263     25
424     28
943     13
379     11
552     26
676     12
278      2
480      7
31       4
1040     2
581      4
430      1
277     16
237     24
878      4
52      25
816     24
131     19
732     30
197     15
        ..
73       1
707      7
404      5
575     24
591      2
565      4
824      4
818     23
1046     4
22      16
745      9
106     25
712      4
693     23
142     17
483     25
208     30
576      4
641      6
69      11
390     19
334      2
888     13
842     23
580     12
504     17
1006     5
376     23
238     15
216      1
Length: 1062, dtype: int64


In [18]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())
dev_batch = (x_dev, y_dev)
start = time.time()
for e in range(config["train_epoch"]):

    t0 = time.time()
    print("Epoch %d start !" % (e + 1))
    for x_batch, y_batch in fill_feed_dict(x_train, y_train, config["batch_size"]):
        return_dict = run_train_step(classifier, sess, (x_batch, y_batch))
        attn = get_attn_weight(classifier, sess, (x_batch, y_batch))
        # plot the attention weight
        # print(np.reshape(attn, (config["batch_size"], config["max_len"])))
    t1 = time.time()

    print("Train Epoch time:  %.3f s" % (t1 - t0))
    dev_acc = run_eval_step(classifier, sess, dev_batch)
    print("validation accuracy: %.3f " % dev_acc)

Epoch 1 start !
Train Epoch time:  487.231 s
validation accuracy: 0.707 
Epoch 2 start !
Train Epoch time:  476.869 s
validation accuracy: 0.781 
Epoch 3 start !
Train Epoch time:  481.681 s
validation accuracy: 0.790 
Epoch 4 start !
Train Epoch time:  497.807 s
validation accuracy: 0.805 
Epoch 5 start !
Train Epoch time:  488.110 s
validation accuracy: 0.815 
Epoch 6 start !
Train Epoch time:  484.989 s
validation accuracy: 0.818 
Epoch 7 start !
Train Epoch time:  478.317 s
validation accuracy: 0.828 
Epoch 8 start !
Train Epoch time:  479.318 s
validation accuracy: 0.837 
Epoch 9 start !
Train Epoch time:  480.892 s
validation accuracy: 0.824 
Epoch 10 start !
Train Epoch time:  470.043 s
validation accuracy: 0.830 
Epoch 11 start !


KeyboardInterrupt: 

In [19]:
print("Training finished, time consumed : ", time.time() - start, " s")
print("Start evaluating:  \n")
cnt = 0
test_acc = 0
for x_batch, y_batch in fill_feed_dict(x_test, y_test, config["batch_size"]):
    acc = run_eval_step(classifier, sess, (x_batch, y_batch))
    test_acc += acc
    cnt += 1

print("Test accuracy : %f %%" % (test_acc / cnt * 100))

Training finished, time consumed :  5208.363984107971  s
Start evaluating:  

Test accuracy : 83.867925 %


In [20]:
preds = []
reals = []
attns = []
for x_batch, y_batch in fill_feed_dict(x_test, y_test,1062, isshuffle=False):
    pred = get_prediction(classifier, sess, (x_batch, y_batch))
    attn = get_attn_weight(classifier, sess, (x_batch, y_batch))
    preds += pred.tolist()
    attns += attn.tolist()
    reals += y_batch.tolist()

In [21]:
test_df = pd.DataFrame({'sent':test_doc,'attn':attns, 'pred':preds, 'real':reals})
test_df

,sent,attn,pred,real
0,ㄷ ㅏ ㅇ ㅡ ㅁ ㅇ ㅔ ㄸ ㅗ ㅎ ㅏ ㄹ ㄲ ㅔ ㅇ ㅛ,"[0.004990768618881702, 0.002199783455580473, 0...",5,5
1,ㄱ ㅡ ㄴ ㅑ ㅇ ⓢ ㅇ ㅣ ㄹ ㅓ ㅎ ㄱ ㅔ ⓢ ㄲ ㅡ ㅌ ㄴ ㅐ ㅁ ㅕ ㄴ ⓢ ...,"[0.003646131372079253, 0.008385280147194862, 0...",24,24
2,ㅈ ㅓ ㅇ ㄱ ㅣ ㄱ ㅕ ㄹ ㅈ ㅐ \ n ㅎ ㅐ ㅈ ㅣ \ n ㅎ ㅐ ㅆ ㅇ ㅡ ...,"[0.004815434571355581, 0.00015554591664113104,...",28,28
3,ㅂ ㅏ ㅇ ㅅ ㅗ ㅇ ㅎ ㅏ ㄹ ㅁ ㅏ ㄴ ㅎ ㅏ ㄴ ㄴ ㅗ ㅌ ㅡ ㅂ ㅜ ㄱ,"[0.012590612284839153, 0.24144107103347778, 0....",8,7
4,ㅂ ㅓ ㄴ ㅍ ㅜ ㅁ ㅎ ㅏ ㄹ ㄹ ㅕ ㄱ ㅜ ㅇ ㅛ,"[9.587463136995211e-05, 0.00021780141105409712...",7,7
5,ㅇ ㅣ ㅇ ㅛ ㅇ ㄱ ㅝ ㄴ ⓢ ㅈ ㅗ ㅇ ㄹ ㅛ ㅎ ㅏ ㄹ ㅕ ㄱ ㅜ ㅇ ㅛ ~ !,"[0.005357284098863602, 0.00219704769551754, 0....",4,4
6,ㅂ ㅣ ㅁ ㅣ ㄹ ㅋ ㅜ ㅍ ㅗ ㄴ ㅇ ㅣ ㅁ ㅝ ㄴ ㄱ ㅏ ㅇ ㅛ,"[6.474051042459905e-05, 5.125881580170244e-05,...",25,25
7,ㅅ ㅏ ㅁ ㅈ ㅟ ⓢ ㅇ ㅔ ㅇ ㅓ ㅋ ㅓ ㄴ ⓢ ㅊ ㅏ ㅈ ㅇ ㅏ ㅈ ㅝ,"[0.12086515873670578, 0.0004427607636898756, 0...",24,24
8,ㅈ ㅓ ㄴ ㅎ ㅘ ㄷ ㅗ ⓢ ㅇ ㅏ ㄴ ㅇ ㅗ ㄱ ㅗ,"[0.0021647275425493717, 0.0005841927486471832,...",17,17
9,ㅂ ㅣ ㅁ ㅣ ㄹ ㅂ ㅓ ㄴ ㅎ ㅗ ⓢ ㅇ ㅓ ㄸ ㅓ ㅎ ㄱ ㅔ ⓢ ㅂ ㅏ ㄲ ㅜ ...,"[0.000259772059507668, 0.0012645629467442632, ...",4,29


In [22]:
test_df['real'] = test_df['real'].apply(lambda x: cols[x])
test_df['pred'] = test_df['pred'].apply(lambda x: cols[x])

In [23]:
f1_score(test_df['real'],test_df['pred'],average='weighted')

0.8368222425366687

In [25]:
test_df[test_df['real'] == test_df['pred']]

,sent,attn,pred,real
0,ㄷ ㅏ ㅇ ㅡ ㅁ ㅇ ㅔ ㄸ ㅗ ㅎ ㅏ ㄹ ㄲ ㅔ ㅇ ㅛ,"[0.004990768618881702, 0.002199783455580473, 0...",DIGITAL_P대화종료,DIGITAL_P대화종료
1,ㄱ ㅡ ㄴ ㅑ ㅇ ⓢ ㅇ ㅣ ㄹ ㅓ ㅎ ㄱ ㅔ ⓢ ㄲ ㅡ ㅌ ㄴ ㅐ ㅁ ㅕ ㄴ ⓢ ...,"[0.003646131372079253, 0.008385280147194862, 0...",DIGITAL_P추천,DIGITAL_P추천
2,ㅈ ㅓ ㅇ ㄱ ㅣ ㄱ ㅕ ㄹ ㅈ ㅐ \ n ㅎ ㅐ ㅈ ㅣ \ n ㅎ ㅐ ㅆ ㅇ ㅡ ...,"[0.004815434571355581, 0.00015554591664113104,...",DIGITAL_P환불문의,DIGITAL_P환불문의
4,ㅂ ㅓ ㄴ ㅍ ㅜ ㅁ ㅎ ㅏ ㄹ ㄹ ㅕ ㄱ ㅜ ㅇ ㅛ,"[9.587463136995211e-05, 0.00021780141105409712...",DIGITAL_P반품문의,DIGITAL_P반품문의
5,ㅇ ㅣ ㅇ ㅛ ㅇ ㄱ ㅝ ㄴ ⓢ ㅈ ㅗ ㅇ ㄹ ㅛ ㅎ ㅏ ㄹ ㅕ ㄱ ㅜ ㅇ ㅛ ~ !,"[0.005357284098863602, 0.00219704769551754, 0....",DIGITAL_P기타faq문의,DIGITAL_P기타faq문의
6,ㅂ ㅣ ㅁ ㅣ ㄹ ㅋ ㅜ ㅍ ㅗ ㄴ ㅇ ㅣ ㅁ ㅝ ㄴ ㄱ ㅏ ㅇ ㅛ,"[6.474051042459905e-05, 5.125881580170244e-05,...",DIGITAL_P쿠폰요청,DIGITAL_P쿠폰요청
7,ㅅ ㅏ ㅁ ㅈ ㅟ ⓢ ㅇ ㅔ ㅇ ㅓ ㅋ ㅓ ㄴ ⓢ ㅊ ㅏ ㅈ ㅇ ㅏ ㅈ ㅝ,"[0.12086515873670578, 0.0004427607636898756, 0...",DIGITAL_P추천,DIGITAL_P추천
8,ㅈ ㅓ ㄴ ㅎ ㅘ ㄷ ㅗ ⓢ ㅇ ㅏ ㄴ ㅇ ㅗ ㄱ ㅗ,"[0.0021647275425493717, 0.0005841927486471832,...",DIGITAL_P전화요청,DIGITAL_P전화요청
10,ㅈ ㅓ ㄴ ㅎ ㅘ ⓢ ㅇ ㅕ ㄴ ㄱ ㅕ ㄹ ㅇ ㅣ ⓢ ㄷ ㅚ ㅇ ㅑ ⓢ ㅁ ㅜ ㄹ ...,"[0.003677999135106802, 0.0004728536296170205, ...",DIGITAL_P전화요청,DIGITAL_P전화요청
11,ㅎ ㅏ ㄴ ㄱ ㅏ ㅈ ㅣ ⓢ ㅅ ㅏ ㅇ ㅍ ㅜ ㅁ ㅇ ㅣ ⓢ ㅁ ㅣ ㅇ ㅣ ㅂ ㄱ ...,"[0.00030335556948557496, 0.0004998987424187362...",DIGITAL_P기타faq문의,DIGITAL_P기타faq문의


In [26]:
idx = np.argsort(-np.array(test_df['attn'][5]))[:10]
idx = sorted(idx)
idx

[3, 6, 7, 8, 9, 10, 12, 13, 16, 19]

In [27]:
pd.Series(list(test_df['sent'][5].replace(' ',''))).loc[idx]

3     ㅛ
6     ㅝ
7     ㄴ
8     ⓢ
9     ㅈ
10    ㅗ
12    ㄹ
13    ㅛ
16    ㄹ
19    ㅜ
dtype: object